# Integrating a model

### Introduction

In the last lesson, we saw how we can use the `flask` library to construct a website that a user visits a certain url, returns data to the user.  In this lesson we'll see how we can 

### Training and Exporting a model

Let's load up some data and train an initial model.

In [2]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
iris_dataset = datasets.load_iris()

X = pd.DataFrame(iris_dataset.data, columns = iris_dataset['feature_names'])
y = iris_dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

In [28]:
X_train[:2]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
53,5.5,2.3,4.0,1.3
141,6.9,3.1,5.1,2.3


In [29]:
model = RandomForestClassifier().fit(X_train, y_train)

In [30]:
model.predict_proba(X_test)[:10]

array([[1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.07, 0.93],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.35, 0.65],
       [0.98, 0.02, 0.  ],
       [0.  , 0.  , 1.  ],
       [0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  ]])

Now that we trained the model, we can export the model with something the joblib library the following.

In [31]:
import joblib
joblib.dump(model, 'rf-model.model')

['rf-model.model']

Notice that we just took our model and saved it as the `rf-model.model` file.  So now, if we ever want to load up and use our already trained model we can do so using joblib.

In [6]:
import joblib
model = joblib.load('rf-model.model')

In [7]:
X_test[1:5]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
3,4.6,3.1,1.5,0.2
113,5.7,2.5,5.0,2.0
12,4.8,3.0,1.4,0.1
24,4.8,3.4,1.9,0.2


In [10]:
model.predict_proba([[5.7, 2.5, 5.0, 2.0]])

array([[0.  , 0.07, 0.93]])

See how that works?

So if we want to make the code just slightly cleaner, we could define a `predict` function that takes in as an argument feature parameters like the sepal length, sepal width, etc, and returns a prediction of what category the flower falls into.

In [13]:
def predict(features):
    return model.predict_proba([features])

In [14]:
x_1 = [5.7, 2.5, 5.0, 2.0]

predict(x_1)

array([[0.  , 0.07, 0.93]])

### Moving to Flask

Now think about how we can use flask to return a prediction.  Remember that the way that the way flask works is that when a user visits a certain path, we run a specific function.

```python
from flask import Flask

app = Flask(__name__)

@app.route('/welcome')
def index():
    return 'hello world'

app.run()
```

So above, when the user visits the `http://127.0.0.1:5000/welcome` the `index` function is run.  And if we add the following code, then we can visit `http://127.0.0.1:5000/make_prediction` and the `predict` function will run.

> **Warning**: The below code won't quite work, but it's close.  

```python
from flask import Flask
import joblib

app = Flask(__name__)

@app.route('/welcome')
def index():
    return 'hello world'

@app.route('/make_prediction')
def predict():
    return model.predict(features)

model = joblib.load('rf-model.model')
app.run(debug = True)
```

Let's unpack what we are doing above.  Starting at the bottom, we see that we use joblib to load up our model, and assign it to a global variable called `model`.  Then above that, we tell flask to call the `predict()` whenever we visit the `make_prediction` path.  We still don't know how we pass through our features so the model can make a prediction, but we'll see that in a bit.

Place the above code into an `app.py` file, and then run `python app.py`.

If we visit the `make_prediction` path, we see the following error.

<img src="./features.png" width="40%">

Ok, so what we would like, is the ability to send in some of our features.  It turns our that we can pass through data when we request a webpage.  Let's use the `request` library to do this.

Let's first show the answer, and then we can explain it.  We'll begin by updating our application code to the following.

```python
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

@app.route('/')
def index():
    return 'hello world'

@app.route('/make_prediction', methods=['POST'])
def predict():
    features = request.get_json()
    prediction = model.predict([features])
    return jsonify(prediction.tolist())

model = joblib.load('rf-model.model')
app.run(debug = True)
```

And then we can make a request to the make_prediction path by using the requests library.

In [41]:
import requests
response = requests.post('http://127.0.0.1:5000/make_prediction', 
              json = [5.7, 2.5, 5.0, 2.0])

In [43]:
response.json()

[2]